In [1]:
import pandas as pd
import os
import json
from tqdm import tqdm
pd.set_option('max_colwidth', 400)

In [2]:
import subprocess
import itertools
from pprint import pprint

In [3]:
import pandas as pd
from scipy import spatial
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample

# Leitura da entrada

In [4]:
def transform_json(diario_json):
    dict_list =  []
    for key, value in json.loads(diario_json).items():
        for v in value:
            v["origin"] = key
            dict_list.append(v)
    return pd.DataFrame(dict_list)

In [5]:
dfs = []
directory = "../saida_segmentacao/AMM/"
for filename in tqdm(list(os.listdir(directory))):
    with open(directory + filename, "r") as jsonfile:
        diario_json = jsonfile.read()
        dfs.append(transform_json(diario_json))
diario_df = pd.concat(dfs)
diario_df.sample(3)

100%|███████████████████████████████████████████████| 1640/1640 [00:30<00:00, 53.00it/s]


,titulo,subtitulo,materia,publicador,id,origin
441,PROCURADORIA GERAL DO MUNICIPIO,LEI MUNICIPAL Nº 2.365 /2018.,"LEI MUNICIPAL Nº 2.365 /2018.Cria no município de Pirapora/MG, a Parada Segura para mulheres após as vinte e uma horas no intinerário dos ônibus de transporte público coletivo urbano. A Câmara Municipal de Pirapora aprovou, e eu, Prefeita Municipal de Pirapora, sanciono a seguinte Lei: Art. 1º Fica determinado que os veículos de transporte público coletivo de linhas regulares do município de P...",Raul Ulysses Rodrigues de Araújo,025C885F,PREFEITURA DE PIRAPORA
197,PREFEITURA MUNICIPAL DE DIVINÓPOLIS,SEMED – TERMO DE ADESÃO PARA TRABALHO VOLUNTÁRIO (A) DE: STEFANY MEDEIROS FERREIRA.,Objeto concessão de estágio junto ao CMEI. RAFAEL NUNES. Prazo 15/12/18. ASS: 18/04/18.,Daniel Felipe da Costa,321103CD,PREFEITURA DE DIVINÓPOLIS
11,DEPARTAMENTO DE LICITAÇÕES,EXTRATO DE CONTRATO TERMO ADITIVO,"MODALIDADE: PREGÃO; OBJETO: Cessão de licença, direito de uso de software, hospedagem na internet pública e de serviços de suporte técnico por tempo determinado pela Câmara Municipal. CONTRATADO:Eduardo Henrique Carlos de Almeida 078.628.026-\n88; CONTRATANTE: Câmara Municipal de Santa Bárbara, CNPJ nº 23.946.247/0001-09; CRITÉRIO DE JULGAMENTO:Menor preço global; O valor total deste instrumen...",Ângela Maria Pereira,17E0C04A,CÂMARA MUNICIPAL DE SANTA BÁRBARA


In [6]:
licitacoes = pd.read_csv("/dados01/workspace/ufmg.m07dcc/M05-dev/indexer/indices/licitacoes/32-licitacoes-das-obras-com_entidades.csv")
licitacoes = licitacoes[['titulo', 'conteudo', 'tipo_documento', 'fonte', 'instancia', "data"]]
licitacoes.columns = ['titulo', 'materia', 'subtitulo', 'publicador', 'origin', "id"]

In [7]:
processos = pd.read_csv("/dados01/workspace/ufmg.m07dcc/M05-dev/indexer/indices/processos/27-tribunais-regionais-federais-trf2-com_metadata.csv")
processos = processos[['conteudo', 'fonte', 'numero_processo', 'titulo', 'tipo_documento', 'instancia']]
processos.columns = ['materia', 'ṕublicador', 'id', 'titulo', 'subtitulo', "origin"]

In [8]:
df = pd.concat([diario_df, licitacoes, processos])
df = df.sample(100000)
df = df[df["materia"].apply(bool)]
df = df[df["materia"].apply(lambda x: type(x) == str)]
df = df[df["materia"].apply(lambda x: len(x) > 20)]

In [9]:
df.head()

,titulo,subtitulo,materia,publicador,id,origin,ṕublicador
318,COMISSÃO DE LICITAÇÃO,AVISO DE LICITAÇÃO PREGÃO Nº 011/2020 PROCESSO Nº 017/2020,"OBJETO: O objetivo da presente licitação é o Registro de Preços para eventual contratação de microempresas - ME, empresas de pequeno porte - EPP ou equiparadas, para aquisição de produtos de padaria e refrigerantes, conforme especificação constante no Anexo I do presente edital. DATA: 20/02/2020 HORÁRIO: 09:00 horas. LOCAL: Rua Norberto Berno, n° 85, Centro, Laranjal, MG, CEP. 36.760-000 FUNDA...",Jéssica Andrade Bani,7922982A,PREFEITURA DE LARANJAL,NaN
255,DEPARTAMENTO DE COMPRAS E LICITAÇÕES,TERMO DE HOMOLOGAÇÃO DE PROCESSO LICITATÓRIO,"PREGÃO PRESENCIAL Nr.: 56/2018 - PR Processo Administrativo: Processo de Licitação: Data do Processo: CNPJ: 18.301.036/0001-70 131/2018 131/2018 12/09/2018 Avenida Laerton Paulinelli, 153 C.E.P.: 35595-000 - Luz - MG TERMO DE HOMOLOGAÇÃO DE PROCESSO LICITATÓRIO O (a) Prefeito, AILTON DUARTE, no uso das atribuições que lhe são conferidas pela legislação em vigor, especialmente sob Lei nº 10.520...",Angela Aparecida Ferreira,CA9AEBE2,PREFEITURA DE LUZ,NaN
496,COMISSÃO DE LICITAÇÃO,HOMOLOGAÇÃO PROCESSO 078/2019,"HOMOLOGAÇÃO Processo n° 078/2019 Pregão Presencial n° 039/2019 Objeto: Registro de Preços para eventual aquisição de pneus novos, devidamente certificados pelo INMETRO, para manutenção dos veículos do Transporte Escolar. DEL REY PNEUS PECAS E EQUIPAMENTOS LTDA - CNPJ: 19.409.408/0001-40 Valor Total de R$9.816,00 (nove mil e oitocentos e dezesseis reais) LUCAS LORENZO COMERCIO E SERVICOS EIRELI...",Adriana Batista de Oliveira,65B5C1B7,PREFEITURA DE PIRAPETINGA,NaN
369,SECRETÁRIA MUNICIPAL DE ADMINISTRAÇÃO,PORTARIA SEMAD N.º 605/2.017,"DISPÕE SOBRE NOMEAÇÃO DE SERVIDOR O Prefeito Municipal de Ribeirão das Neves, no uso de suas atribuições legais que lhe confere o inciso VI, do artigo 95 e com base na alínea “a”, inciso II, do artigo 58, ambos da Lei Orgânica do Município, RESOLVE: ART.1º - Nomear o senhor ANDERSON BARBOSA DE SIQUEIRA para exercer o cargo em comissão de livre nomeação e exoneração de GERENTE DE PROJETOS, da S...",Elienai Pereira da Silva,EA688310,PREFEITURA DE RIBEIRÃO DAS NEVES,NaN
352,SECRETARIA MUNICIPAL DE ADMINISTRAÇÃO,EXTRATO DE CONTRATO,"ATA DE REGISTRO DE PREÇOS 174/2019 Decorrência: PROCESSO LICITATÓRIO Nº 117/2019 – PREGÃO Nº 78/2019 Objeto:REGISTRO DE PREÇOS PARA FUTURAS E EVENTUAIS CONTRATAÇÕES DE EMPRESA ESPECIALIZADA EM PRESTAÇÃO DE SERVIÇOS DE ALINHAMENTO, BALANCEAMENTO, CAMBAGEM E CASTER PARA MANUTENÇÃO DOS VEÍCULOS DA FROTA MUNICIPAL, DE ACORDO COM O SOLICITADO DURANTE O ANO DE 2.019. Contratado: GILSON JOSE FERREIRA...",Patricia Sarkis Carneiro Abrahao,71D8EB52,PREFEITURA DE POÇO FUNDO,NaN


# Modelo

In [10]:
from sentence_transformers import SentenceTransformer, models
from torch import nn

word_embedding_model = models.Transformer('neuralmind/bert-base-portuguese-cased', max_seq_length=250)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [11]:
model.encode("CHAMADA PARA CADASTRAMENTO DE FORNECEDORES A Prefeitura Municipal de Guarani, Minas Gerais, torna público, que o Registro Cadastral está permanentemente aberto para o ingresso de novas empresas interessadas em fazer sua inscrição para o ano de 2014, e convoca as empresas já inscritas no Cadastro de Fornecedores, que estiverem com o Certificado de Registro Cadastral fora do prazo de vigência, a")

array([ 1.42166503e-02, -1.56924382e-01,  1.08171649e-01,  1.38051882e-01,
        1.58805102e-01,  9.85571519e-02, -3.14570010e-01,  6.58817217e-02,
       -1.79993138e-02, -3.16501558e-01,  9.20951590e-02,  1.51275963e-01,
       -1.41864523e-01,  2.96851158e-01, -1.33947715e-01, -8.91353860e-02,
        3.21372360e-01, -6.97905645e-02,  1.21856682e-01,  5.10391407e-02,
       -1.95272952e-01,  2.04893574e-01, -2.39906181e-02, -2.75616854e-01,
       -1.16283633e-01,  4.35893595e-01, -2.42359996e-01, -8.64645690e-02,
       -1.89686686e-01, -3.33870828e-01, -5.32129519e-02,  3.89226168e-01,
       -9.28680450e-02,  4.04962182e-01,  4.06588800e-02,  3.71585861e-02,
        2.31454089e-01, -9.61590111e-02, -3.18237357e-02,  1.29023060e-01,
        1.60464764e-01,  1.93644255e-01, -2.52366960e-01, -1.11598447e-01,
        2.53539458e-02, -4.46110189e-01, -8.21894854e-02,  1.78402126e-01,
        7.46348780e-03, -3.81555371e-02,  1.92299858e-01,  3.54876905e-03,
       -9.42660794e-02,  

# Candidatos

In [12]:
from gensim import corpora
from gensim.summarization import bm25

In [13]:
corpus = list(df["materia"])
texts = [doc.split() for doc in corpus]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
bm25_obj = bm25.BM25(corpus)

def get_candidates(df, query, bm25_obj, n = 100):
    query_doc = dictionary.doc2bow(query.split())
    df["bm25_scores"] = bm25_obj.get_scores(query_doc)
    return df.sort_values(by="bm25_scores").iloc[-1 * n:]

In [14]:
def get_similarities(sentence, query, n=5):
    similarities = []
    output= model.encode([query, sentence])
    value = 1 - spatial.distance.cosine(output[0], output[1])
    similarities.append((sentence, value))

    similarities = sorted(similarities, key= lambda x: x[1])[-n:]
    return similarities

def get_score(similarities):
    score = sum(i[1] for i in similarities)
    return score

def get_sentences_scores(similarities):
    score = [round(i[1], 4) for i in similarities]
    return score

def get_top_sentences(similarities):
    top_sentences = [i[0] for i in similarities]
    return top_sentences

def apply_columns_df(df_candidates, query):
    df_candidates["similarities"] = df_candidates["materia"].apply(lambda sentences: get_similarities(sentences, query))
    df_candidates["sum_scores"] = df_candidates["similarities"].apply(lambda similarities: get_score(similarities))
    df_candidates["sentences_scores"] = df_candidates["similarities"].apply(lambda similarities: get_sentences_scores(similarities))
    df_candidates["top_sentences"] = df_candidates["similarities"].apply(lambda similarities: get_top_sentences(similarities))
    df_candidates["query"] = query
    return df_candidates

In [15]:
queries = ['Ação', 'Ação civil pública', 'Ação originária', 'Ação cautelar', 'Ação rescisória', 'Ação trabalhista', 'Acidente de trabalho', 'Acórdão', 'Acordo', 'Agravo', 'agravo de instrumento', 'agravo de petição', 'Alçada', 'A quo', 'Arbitragem', 'Arquivado', 'Arquivo provisório', 'Audiência de instrução e julgamento', 'Autônomo', 'Autos', 'Autuação', 'Aviso-prévio', 'Avulso', 'Bis in idem', 'Caput', 'CTPS', 'Carga', 'Carta precatória', 'Carta rogatória', 'Carta de sentença', 'Celetista', 'Consolidado', 'Certidão de objeto e pé', 'Certificado digital', 'Citação', 'Coisa julgada', 'Cipa', 'CNDT', 'Comissão de Conciliação Prévia', 'Conciliação', 'Composição', 'Conclusos', 'Conflito de competência', 'Contrarrazões', 'Contribuição sindical ', 'Assistencial', 'Confederativa', 'Cooperativa', 'Correição', 'Custas', 'DSR', 'Decadência', 'Décimo terceiro salário', 'Decisão interlocutória', 'Declaração de pobreza', 'Desembargador', 'Deserção', 'Despacho', 'Dilação', 'Diligência', 'Dissídio', 'Dissídio coletivo', 'Dissídio individual', 'Distribuição', 'Edital', 'E-doc', 'Efeito suspensivo', 'Embargos', 'de declaração', 'à execução que', 'de terceiro', 'Ementa', 'Empregado', 'Empregador', 'Execução', 'Ex nunc', 'Ex officio', 'Exordial', 'Ex tunc', 'Estatutário', 'FAT', 'Férias', 'FGTS', 'Foro', 'Gorjeta', 'GRU', 'Habeas corpus', 'Habeas data', 'Hasta Pública', 'Homologação', 'Honorários', 'Hora extra', 'Impedimento', 'Instância', 'grau de jurisdição', 'Intempestivo', 'Jornada de trabalho', 'Juiz', 'Juiz classista', 'Jurisdição', 'Jurisprudência', 'Juros de mora', 'Jus postulandi', 'Justa causa', 'Justiça gratuita', 'Laudo', 'Leilão judicial', 'Licitação', 'Lide', 'Liminar', 'Liquidação', 'Litigante de má fé', 'Litisconsórcio', 'Litispendência', 'Lockout', 'Mandado judicial', 'Mandado de segurança', 'Medida cautelar', 'Mérito da ação', 'Ministério do Trabalho', 'Ministério Público do Trabalho', 'Normas regulamentares', 'Notificação', 'Obreiro', 'Oficial de justiça', 'Ônus da prova', 'Orientação jurisprudencial', 'Ouvidoria', 'PAT', 'Parecer', 'Partes', 'Penhora', 'Perícia', 'Petição', 'Plantão judiciário', 'Portaria', 'Praça pública', 'Precad', 'Precatório', 'Preliminar', 'Preposto', 'Prescrição', 'Prioridade', 'Processo', 'Processos pendentes', 'Procuração', 'ad judicia', 'Protelatório', 'Procrastinatório', 'Provimento', 'PJe-JT', 'Recesso', 'Reclamado', 'Reclamante', 'Recolhimento previdenciário', 'Recurso ordinário', 'Recurso', 'ex officio', 'Redução a termo', 'Relator', 'Relatório', 'Responsabilidade solidária', 'Subsidiária', 'Revelia', 'Revisor', 'Rito', 'Rito ou procedimento sumário', 'Rito ou procedimento sumaríssimo', 'Salário', 'Segredo de justiça', 'Seguro desemprego', 'Sentença', 'Sessão de julgamento', 'Sindicato', 'Sisdoc', 'Sobrestado', 'Sobreaviso', 'Substabelecimento', 'Sucumbência', 'Súmula', 'Suspeição', 'Sustentação oral', 'SRTE', 'Testemunha', 'Transação', 'Trânsito em julgado', 'Turma', 'Tutela', 'Ulterior', 'Vara do Trabalho', 'Vínculo empregatício', 'Voto']

In [16]:
dfs = []
for query in tqdm(queries,  position=0, leave=True):
    df_candidates = get_candidates(df.copy(deep=True), query, bm25_obj)
    df_candidates = apply_columns_df(df_candidates, query)
    df_rank = df_candidates.sort_values(by="sum_scores", ascending=False).iloc[:5]
    df_rank = df_rank[['materia', 'id', 'origin','bm25_scores', 'sum_scores', 'query']]
    dfs.append(df_rank)

feedback_df = pd.concat(dfs)

100%|█████████████████████████████████████████████████| 192/192 [07:10<00:00,  2.24s/it]


In [17]:
feedback_df = feedback_df.sort_values(by="sum_scores", ascending=False).groupby("query").head()


In [18]:
feedback_df.sort_values(by="sum_scores", ascending=False).head(20)

,materia,id,origin,bm25_scores,sum_scores,query
483,A CPL torna público que tomará as providências para revogar o certame conforme disposto na ata de julgamento. Abre-se vista e prazo recursal.,992EFF9D,PREFEITURA DE VESPASIANO,7.801494,0.642939,Audiência de instrução e julgamento
241,Processo Licitatório nº 156/2019 – Concorrência Nº 09/2019– Não acudiram interessados. Licitação declarada deserta. CPL.,14B226FF,PREFEITURA DE ITATIAIUÇU,4.766011,0.640799,Licitação
170,Contratado NAYARA ISABELA BORGES - a partir de 28 de outubro de 2019.- Justificativa: Término de Vínculo gravídico.,BC5DA05E,PREFEITURA DE DIVINÓPOLIS,14.593905,0.634288,Vínculo empregatício
173,Torna sem efeito a Portaria que menciona.,EE4469F6,PREFEITURA DE FORMIGA,4.536577,0.631541,Portaria
255,Processo Licitatório nº 195/2017 - Pregão Presencial Nº 139/2017 – Não acudiram interessados. Licitação declarada deserta. Pregoeira.,ACC8066A,PREFEITURA DE ITATIAIUÇU,4.718523,0.624709,Licitação
554,Fica INDEFERIDO o recurso manejado no Processo Administrativo de Tombamento nº023/2018.,835D2127,PREFEITURA MUNICIPAL DE VISCONDE DO RIO BRANCO,5.965778,0.609287,agravo de petição
381,Processo Licitatório nº 022/2018 - Pregão Presencial Nº 013/2018 – Não acudiram interessados. Licitação declarada deserta. Pregoeira.,1506BAE4,PREFEITURA DE ITATIAIUÇU,4.718523,0.604523,Licitação
148,"OBJETO: Regulamentação do serviço a ser prestado, a pedido pelo voluntário, não gerando nenhum vinculo empregatício nos termos da legislação regente. Com jornada de (04) horas. PRAZO: 30/06/2019. ASS: 21/04/2019.",3B2CBB04,PREFEITURA DE DIVINÓPOLIS,10.139174,0.602580,Vínculo empregatício
585,"Comunicamos que encontra-se disponível no site a Ata de Julgamento proferida em dia 10/07/2018. Abre-se vista e prazo p/ recurso. LECIA SOARES, Presidente CPL.",617818B7,PREFEITURA DE VESPASIANO,7.763937,0.600306,Audiência de instrução e julgamento
554,Aviso de suspensão do Pregão Eletrônico nº. SAAE/SLO-037/2019 Objeto: registro de preços para futuras e eventuais aquisições de bombas e motores. Motivo: retificação do Edital Oportunamente será marcada nova data para a sessão pública,14129E42,PREFEITURA DE SÃO LOURENÇO,4.189807,0.599818,Edital


## Feedback data

In [23]:
feedback_data = []

for query in feedback_df["query"].unique():
    feedback_line = {}
    feedback_line["text"] = query
    feedback_line["_id"] = query
    feedback_line["corresponding"] = []
    for idx, row in feedback_df[feedback_df["query"] == query].iterrows():
        feedback_line["corresponding"].append({"text": row["materia"].encode('utf-8').decode("utf-8"), "_id": row["id"]})
    feedback_data.append(feedback_line)

In [22]:
json.dumps(feedback_data, ensure_ascii=False)

NameError: name 'feedback_data' is not defined

In [ ]:
with open("feedback.json", "w") as output:
    output.write(json.dumps(feedback_data, ensure_ascii=False))

# Feedback distribution

In [20]:
queries = ['Ação', 'Ação civil pública', 'Ação originária', 'Ação cautelar', 'Ação rescisória', 'Ação trabalhista', 'Acidente de trabalho', 'Acórdão', 'Acordo', 'Agravo', 'agravo de instrumento', 'agravo de petição', 'Alçada', 'A quo', 'Arbitragem', 'Arquivado', 'Arquivo provisório', 'Audiência de instrução e julgamento', 'Autônomo', 'Autos', 'Autuação', 'Aviso-prévio', 'Avulso', 'Bis in idem', 'Caput', 'CTPS', 'Carga', 'Carta precatória', 'Carta rogatória', 'Carta de sentença', 'Celetista', 'Consolidado', 'Certidão de objeto e pé', 'Certificado digital', 'Citação', 'Coisa julgada', 'Cipa', 'CNDT', 'Comissão de Conciliação Prévia', 'Conciliação', 'Composição', 'Conclusos', 'Conflito de competência', 'Contrarrazões', 'Contribuição sindical ', 'Assistencial', 'Confederativa', 'Cooperativa', 'Correição', 'Custas', 'DSR', 'Decadência', 'Décimo terceiro salário', 'Decisão interlocutória', 'Declaração de pobreza', 'Desembargador', 'Deserção', 'Despacho', 'Dilação', 'Diligência', 'Dissídio', 'Dissídio coletivo', 'Dissídio individual', 'Distribuição', 'Edital', 'E-doc', 'Efeito suspensivo', 'Embargos', 'de declaração', 'à execução que', 'de terceiro', 'Ementa', 'Empregado', 'Empregador', 'Execução', 'Ex nunc', 'Ex officio', 'Exordial', 'Ex tunc', 'Estatutário', 'FAT', 'Férias', 'FGTS', 'Foro', 'Gorjeta', 'GRU', 'Habeas corpus', 'Habeas data', 'Hasta Pública', 'Homologação', 'Honorários', 'Hora extra', 'Impedimento', 'Instância', 'grau de jurisdição', 'Intempestivo', 'Jornada de trabalho', 'Juiz', 'Juiz classista', 'Jurisdição', 'Jurisprudência', 'Juros de mora', 'Jus postulandi', 'Justa causa', 'Justiça gratuita', 'Laudo', 'Leilão judicial', 'Licitação', 'Lide', 'Liminar', 'Liquidação', 'Litigante de má fé', 'Litisconsórcio', 'Litispendência', 'Lockout', 'Mandado judicial', 'Mandado de segurança', 'Medida cautelar', 'Mérito da ação', 'Ministério do Trabalho', 'Ministério Público do Trabalho', 'Normas regulamentares', 'Notificação', 'Obreiro', 'Oficial de justiça', 'Ônus da prova', 'Orientação jurisprudencial', 'Ouvidoria', 'PAT', 'Parecer', 'Partes', 'Penhora', 'Perícia', 'Petição', 'Plantão judiciário', 'Portaria', 'Praça pública', 'Precad', 'Precatório', 'Preliminar', 'Preposto', 'Prescrição', 'Prioridade', 'Processo', 'Processos pendentes', 'Procuração', 'ad judicia', 'Protelatório', 'Procrastinatório', 'Provimento', 'PJe-JT', 'Recesso', 'Reclamado', 'Reclamante', 'Recolhimento previdenciário', 'Recurso ordinário', 'Recurso', 'ex officio', 'Redução a termo', 'Relator', 'Relatório', 'Responsabilidade solidária', 'Subsidiária', 'Revelia', 'Revisor', 'Rito', 'Rito ou procedimento sumário', 'Rito ou procedimento sumaríssimo', 'Salário', 'Segredo de justiça', 'Seguro desemprego', 'Sentença', 'Sessão de julgamento', 'Sindicato', 'Sisdoc', 'Sobrestado', 'Sobreaviso', 'Substabelecimento', 'Sucumbência', 'Súmula', 'Suspeição', 'Sustentação oral', 'SRTE', 'Testemunha', 'Transação', 'Trânsito em julgado', 'Turma', 'Tutela', 'Ulterior', 'Vara do Trabalho', 'Vínculo empregatício', 'Voto']

In [21]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [27]:
import random
random.shuffle(queries)

In [32]:
for sublist, name in zip(chunks(queries, int(len(queries) / 3)), ["rennan", "rodrygo", "rafael"]):
    feedback_data_sublist = [entry for entry in feedback_data if entry["text"] in sublist]
    with open(f"feedback_{name}.json", "w") as output:
        output.write(json.dumps(feedback_data_sublist, ensure_ascii=False))